# Exemplo Análise de textos - movie review

O arquivo *movie_review* armazena uma coleção de avaliações com os respectivos sentimentos, sendo 1 - positivo e 0 - negativo.

O arquivo foi baseado neste dataset: http://ai.stanford.edu/~amaas/data/sentiment/

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(tm)
library(SnowballC)
library(wordcloud)
library(text2vec)
library(glmnet)
library(caret)

## Carregar dados (*Corpus*)

In [ ]:
data("movie_review")

In [ ]:
movie_review %>% 
    head()

## Pré processamento

As funções de expressão regulares auxiliam na tarefa de pré processamento.

Alguns exemplos são encontrados aqui: https://stat.ethz.ch/R-manual/R-devel/library/base/html/regex.html

In [ ]:
texto_preprocessado <- movie_review$review %>% 
    gsub('[[:cntrl:]]', "", .) %>% # retira caracteres de controle
    gsub("http\\S+\\s*", "", .) %>% # retira caracteres relacionados ao html
    gsub("\\d+", "", .) %>% # retira caracteres numericos 
    gsub("[^[:graph:]]", " ", .) %>% # retira caracteres gráficos
    str_replace_all("[^[:alnum:]]", " ") %>% # retira símbolos não alfanuméricos
    str_replace_all("\\s+", " ") %>% # retira múltiplos espaços em branco
    str_to_lower() # texto para minusculo

In [ ]:
texto_preprocessado %>% 
    head()

Para que este processo não seja tão repetitivo, pode-se criar funções que podem chamadas quando necessário.

In [ ]:
preprocessamento_texto <- function(texto) {
  
    texto %>% 
        gsub('[[:cntrl:]]', "", .) %>% # retira caracteres de controle
        gsub("http\\S+\\s*", "", .) %>% # retira caracteres relacionados ao html
        gsub("\\d+", "", .) %>% # retira caracteres numericos 
        gsub("[^[:graph:]]", " ", .) %>% # retira caracteres gráficos
        str_replace_all("[^[:alnum:]]", " ") %>% # retira símbolos não alfanuméricos
        str_replace_all("\\s+", " ") %>% # retira múltiplos espaços em branco
        str_to_lower() # texto para minusculo
    
}

## Tabela de sinônimos

A tabela de sinônimos é uma etapa importante, porém ela pode ser desenvolvida durante todo o projeto de análise de texto.

## Tokens

Nesta etapa ocorre a divisão do conjunto de caracteres em uma lista de termos:

In [ ]:
# exemplo
stringr::str_split( texto_preprocessado[1], 
                   pattern = stringr::boundary("word") )

Também pode-se criar funções que podem chamadas quando necessário.

In [ ]:
preprocessamento_token <- function( texto ) {
  
  stringr::str_split( texto, pattern = stringr::boundary("word") )
  
}

## Matriz de documentos e termos

Neste exemplo iremos criar uma matriz pelo método *Bag of Words* (TF-DF), dado que desejamos criar uma nuvem de palavras. Na etapa de desenvolver o modelo de sentimento serão apresentados os métodos *one hot encoding* e TF-IDF.

O processo de criar a matriz é muito oneroso computacionalmente, tanto de processamento quanto de memória. Desta forma, iremos utilizar algumas funções do pacote *text2vec* (http://text2vec.org) que otimiza este trabalho.

In [ ]:
# criamos uma função de iteração
iterador <- itoken( movie_review$review, # textos
                    preprocessor = preprocessamento_texto, # funcao de pre processamento
                    tokenizer = preprocessamento_token, # divisao dos termos
                    ids = movie_review$id, # id do texto
                    progressbar = FALSE )

A matriz precisa ser construída a partir de um vocabulário

In [ ]:
vocabulario <- create_vocabulary( it = iterador, 
                                  ngram = c(1, 1), # combinacao de palavras
                                  stopwords = stopwords("english") # palavras comuns que podem ser desconsideradas 
                                )

In [ ]:
# stopwords
stopwords("english")

In [ ]:
vocabulario

Como pode ser visto, muitos termos não fazem sentido estar nas análises. Portanto, torna-se importante podar o vocabulário.

In [ ]:
vocabulario <- prune_vocabulary( vocabulario, 
                                 term_count_min = 10,  
                                 doc_proportion_min = 0.05, 
                                 doc_proportion_max = 0.9 )

In [ ]:
vocabulario

Também é observados termos com somente uma ou duas letras/símbolos. Desta forma, também é importante ajustar estas ocorrências.

In [ ]:
vocabulario %<>% 
    filter( nchar(term) >= 4 )

In [ ]:
vocabulario

Após estas etapas, a matriz de termos pode ser construída.

In [ ]:
document_term_matrix <- create_dtm( iterador, 
                                    vocab_vectorizer( vocabulario ) )

In [ ]:
dim(document_term_matrix)

In [ ]:
document_term_matrix

## Análises

### Wordcloud

Utilizando-se o vocabulário criado anteriormente, podemos criar a nuvem de palavras.

In [ ]:
# 50 termos mais frequentes
vocabulario %>% 
    arrange( desc(term_count) )  %>% 
    head( 50 )

In [ ]:
wordcloud( words = vocabulario$term, 
           freq = vocabulario$term_count, 
           max.words =  50,
           colors = c("blue","red") )

### Análise de sentimentos

Uma forma de realilzar uma análise de sentimentos é aplicar um *Machine Learning* supervisionado utilizando a matriz de documentos e termos. No caso do nosso exemplo *movie reviews*, podemos aplicar um modelo de regressão logística, uma vez que a classificação é binária ( 1 - positivo, 0 - negativo).

In [ ]:
movie_review %>% 
    count( sentiment )

Vimos anteriormente que a matriz de documentos e termos foi construída utilizando o método *Bag of words* e armazenada no objeto *document_term_matrix*. Iremos criar outras duas matrizes utilizando os métodos *one hot enconding* e TF-IDF.

- TF-IDF: *Inverse Document Frequency*

In [ ]:
# gera o método de transformação TF-IDF
tfidf = TfIdf$new()
# transforma a matriz
document_term_matrix_idf <- fit_transform(document_term_matrix, tfidf)

In [ ]:
document_term_matrix_idf

- *One hot encoding*

In [ ]:
# transforma a matriz
document_term_matrix_onehot <- (document_term_matrix > 0) * 1

In [ ]:
document_term_matrix_onehot

Após definirmos as matrizes, podemos aplicar os modelos de regressão logística. Como são muitos termos, não é recomendado verificar os coeficientes.

In [ ]:
model_onehot <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix_onehot, 
       family = 'binomial')

In [ ]:
model_tfidf <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix_idf, 
       family = 'binomial')

In [ ]:
model_bagwords <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix, 
       family = 'binomial')

- Podemos avaliar os modelos

In [ ]:
pred_model_onehot <- predict( model_onehot, s=0.01, document_term_matrix_onehot, type = 'response' )
pred_model_tfidf <- predict( model_tfidf, s=0.01, document_term_matrix_idf, type = 'response')
pred_model_bagwords <- predict( model_bagwords, s=0.01, document_term_matrix, type = 'response')

In [ ]:
# metodo onehot
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_onehot,
                                   threshold = 0.5 )

In [ ]:
# metodo tf-idf
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_tfidf,
                                   threshold = 0.5 )

In [ ]:
# metodo bag of words
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_bagwords,
                                   threshold = 0.5 )

usando o pacote caret

In [ ]:
pred_model_onehot_bin <- as.factor(ifelse( pred_model_onehot > 0.5, 1, 0 ))
pred_model_tfidf_bin <- as.factor(ifelse( pred_model_tfidf > 0.5, 1, 0 ))
pred_model_bagwords_bin <- as.factor(ifelse( pred_model_bagwords > 0.5, 1, 0 ))

In [ ]:
caret::confusionMatrix( as.factor(movie_review$sentiment),
                        pred_model_onehot_bin,
                        positive = '1' )

In [ ]:
caret::confusionMatrix( as.factor(movie_review$sentiment),
                        pred_model_tfidf_bin,
                        positive = '1' )

In [ ]:
caret::confusionMatrix( as.factor(movie_review$sentiment),
                        pred_model_bagwords_bin,
                        positive = '1' )